### Notebook to perform gerbil related actions

#### Submit Gerbil experiments individually for each language

**Set variables**

In [ ]:
# imports
import os
import time
from tqdm import tqdm
import sys
sys.path.append('../code/')
from components.Gerbil import Gerbil

In [ ]:
# variables
# reference_file = "../datasets/qald9plus/wikidata/qald_9_plus_test_wikidata.json"
# reference_file_label = "QALD 9 Plus - Test"
reference_file = "../datasets/qald10/qald_10.json"
reference_file_label = "QALD 10"

# prediction_root = "../predictions_qald9plus_test_multilingual"
prediction_root = "../predictions_qald10_multilingual/"

prediction_models = []

# Use the loop below if you want all directories to be evaluated by Gerbil
for file in os.listdir(prediction_root):
    d = os.path.join(prediction_root, file)
    if os.path.isdir(d):
        prediction_models.append(file)

# languages = ["en","de","ru","fr","lt","ba","be","uk","zh","ja","es"]
languages = ["en","de","ru","zh"]

**Submit experiments**

In [ ]:
# Gerbil evaluation
gerbil_dict = dict()
# for each prediction model
for pred_model in tqdm(prediction_models):
    print('Processing predictions for %s' % pred_model)
    pred_path = f"{prediction_root}/{pred_model}"
    # Gerbil experiments
    gerbil_lang_dict = dict()
    
    for language in languages:
        print('Uploading results to Gerbil for %s' % language)
        model_name = pred_model
        # Initializing Gerbil object
        gerbil = Gerbil()
        gerbil.add_ref_file(reference_file_label, reference_file)
        
        output_file = f"{pred_path}/{language}.json"
        
        gerbil.add_pred_file(f"{model_name}-{language}", output_file, language)
        response = gerbil.submit_experiment(language)
        print(response)
        exp_uri_file = f"{pred_path}/exp_uri_{language}"
        # Write the experiment URI to a file
        if response and response.text:
            print('Gerbil response %s' % response.text)
            exp_id = response.text
            exp_id = "https://gerbil-qa.aksw.org/gerbil/experiment?id=" + exp_id.strip()
            with open(exp_uri_file, 'w') as exp_det:
                exp_det.write(exp_id + '\n')
        # Save gerbil object for export later
        gerbil_lang_dict[language] = gerbil
        time.sleep(10)
    gerbil_dict[pred_path] = gerbil_lang_dict

**Export experiment results**

In [ ]:
# Export Gerbil results
for pred_path in gerbil_dict:
    gerbil_lang_dict = gerbil_dict[pred_path]
    # Export the Gerbil result files
    for key in gerbil_lang_dict:
        result_file_name = f"{pred_path}/result_{key}.csv"
        gerbil : Gerbil = gerbil_lang_dict[key]
        # Export results to a csv
        gerbil.export_results(result_file_name)

**Display formatted results**

In [ ]:
# for each prediction model
for pred_model in prediction_models:
    pred_path = f"{prediction_root}/{pred_model}"
    for language in languages:
        result_file_name = f"{pred_path}/result_{language}.csv"
        uri_file_name = f"{pred_path}/exp_uri_{language}"
        with open(result_file_name, 'r') as rf, open(uri_file_name, 'r') as uf:
            header = rf.readline()
            results = rf.readline().rstrip().split(',')
            uri = uf.readline().rstrip()
            print(pred_model + '\t' + ('\t'.join(results)) + '\t' + uri)